# Pinhole camera model

![Video](https://www.youtube.com/watch?v=qByYk6JggQU)

<iframe width="560" height="315" src="https://www.youtube.com/watch?v=qByYk6JggQU" frameborder="0" allowfullscreen></iframe>

Extrinsic Parameters

Intrinsic Parameters

![camera calibration](https://github.com/lukestairwalker/OpenCV/blob/main/images/flower.jpg)


### Question: How to calculate 3D points from the real world to 2D points in the camera coordinate system. 

Using the camera calibration matrix.

In [7]:
import cv2
import numpy as np

# Beispielwerte einer typischen Kamerakalibrierung (in Pixeln)
# Diese Werte bekommst du normalerweise durch ein Kalibrier-Skript mit Schachbrett-Bildern

# Intrinsische Kamera-Matrix
K = np.array([
    [1000, 0, 640],   # fx, s, cx
    [0, 1000, 360],   # 0, fy, cy
    [0, 0, 1]
])

# Verzerrungskoeffizienten (hier beispielhaft, wichtig für reale Kameras)
dist_coeffs = np.array([0.1, -0.05, 0, 0, 0])

# Simulierter 3D-Punkt in Kamerakoordinaten (z.B. 2 Meter vor der Kamera)
point_3D = np.array([[0.5, 0.2, 2.0]], dtype=np.float32)  # X, Y, Z

# Projektionsfunktion von OpenCV
image_points, _ = cv2.projectPoints(point_3D, rvec=np.zeros(3), tvec=np.zeros(3), cameraMatrix=K, distCoeffs=dist_coeffs)

print("2D-Punkt im Bild:", image_points[0][0])

2D-Punkt im Bild: [891.7468  460.69873]
